In [46]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import datetime
import os

In [47]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales/'
df_train = pd.read_csv(path + '/data/trainclean.csv', sep=';')
df_train['date'] = pd.to_datetime(df_train['date'])
df_test = pd.read_csv(path + '/data/testclean.csv', sep=';')
df_test['date'] = pd.to_datetime(df_test['date'])

df = pd.concat([df_train, df_test], axis=0)
df.head()

,ts_id,date,id,bu,famid,sales,onprom
0,1_0,2013-01-02,1782,1,0,2.0,0
1,1_0,2013-01-03,3564,1,0,3.0,0
2,1_0,2013-01-04,5346,1,0,3.0,0
3,1_0,2013-01-05,7128,1,0,5.0,0
4,1_0,2013-01-06,8910,1,0,2.0,0


In [48]:
df_f_d = df[['date']].drop_duplicates()
df_f_d['day'] = df_f_d['date'].dt.day
df_f_d['week'] = df_f_d['date'].dt.isocalendar().week
df_f_d['month'] = df_f_d['date'].dt.month
df_f_d['year'] = df_f_d['date'].dt.year
df_f_d['yearmoment'] = (df_f_d['year'] - df_f_d['year'].min())
df_f_d['weekofmonth'] = df_f_d['day'].apply(lambda x: np.ceil(x/7))

df_f_d['dayofweek'] = df_f_d['date'].dt.dayofweek
df_f_d['weekend'] = (df_f_d['dayofweek']>=5)

df_f_d['cosweek'] = np.cos(df_f_d['week'])
df_f_d['sinweek'] = np.sin(df_f_d['week'])

df_f_d.tail()

df_hol_event = pd.read_csv(path + '/data/holidays_events.csv')
hol_ev_keep = ['Holiday', 'Additional', 'Event']
df_hol_event = df_hol_event[df_hol_event['type'].isin(hol_ev_keep)][['date']]
df_hol_event['date'] = pd.to_datetime(df_hol_event['date'])
df_hol_event = df_hol_event.drop_duplicates()

df_hol_event['hol'] = 1

df_f_d_h = df_f_d.merge(df_hol_event, on=['date'], how='left')
df_f_d_h['hol'] = df_f_d_h['hol'].fillna(0).astype(int)
df_f_d_h['hol_before_1'] = df_f_d_h['hol'].shift(1).fillna(0).astype(int)
df_f_d_h['hol_before_2'] = df_f_d_h['hol'].shift(2).fillna(0).astype(int)
df_f_d_h['hol_before_3'] = df_f_d_h['hol'].shift(3).fillna(0).astype(int)
df_f_d_h['hol_after_1'] = df_f_d_h['hol'].shift(-1).fillna(0).astype(int)

df_f_d_h.head()

df_oil = pd.read_csv(path + '/data/oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil.columns = ['date', 'prixoil']
df_f_d_h_o = df_f_d_h.merge(df_oil, on=['date'], how='left')
df_f_d_h_o['prixoil'] = df_f_d_h_o['prixoil'].fillna(method='ffill').fillna(method='bfill')

df_f_d_h_o.to_csv('data/fe/date_holidays_oil.csv', index=False, sep=';')
df_f_d_h_o.head()


df_store = pd.read_csv(path + '/data/stores.csv')

df_store['typeid'] = pd.factorize(df_store['type'])[0]
df_store['cityid'] = pd.factorize(df_store['city'])[0]
df_store['stateid'] = pd.factorize(df_store['state'])[0]

df_store_infos = df_store[['store_nbr', 'city', 'state', 'type', 'typeid', 'cityid', 'stateid', 'cluster']].drop_duplicates()
df_store_infos.to_csv('data/df_store_infos.csv', index=False, sep=';')

df_transaction = pd.read_csv(path + '/data/transactions.csv')
df_transaction.columns = ['date', 'bu', 'transactions']
df_transaction['date'] = pd.to_datetime(df_transaction['date'])

df_store_transactions = df[['date', 'bu']].drop_duplicates() \
    .merge(df_transaction, how='left', on=['date', 'bu'])

df_store_transactions['transactions'] = df_store_transactions['transactions'].fillna(0)

df_store_transactions_hol = df_store_transactions.merge(df_f_d_h[['date', 'hol']], on=['date'], how='left')
df_store_transactions_hol['isclosed'] = 0
df_store_transactions_hol.loc[(df_store_transactions_hol['hol'] == 1) & (df_store_transactions_hol['transactions'] == 0),
                              'isclosed'] = 1

df_store_transactions = df_store_transactions_hol.drop('hol', axis=1)

df_store = df_store[['store_nbr', 'typeid', 'cityid', 'stateid', 'cluster']]
df_store.columns = ['bu', 'typeid', 'cityid', 'stateid', 'cluster']

df_store_transactions = df_store_transactions.merge(df_store, how='left', on=['bu'])
df_store_transactions = df_store_transactions.drop_duplicates()
df_store_transactions.to_csv('data/fe/stores_details_transactions.csv', index=False, sep=';')

df_store_transactions.head()

,date,bu,transactions,isclosed,typeid,cityid,stateid,cluster
0,2013-01-02,1,2111.0,0,0,0,0,13
1,2013-01-03,1,1833.0,0,0,0,0,13
2,2013-01-04,1,1863.0,0,0,0,0,13
3,2013-01-05,1,1509.0,0,0,0,0,13
4,2013-01-06,1,520.0,0,0,0,0,13


In [49]:
def calculate_lags(df, col, lags, shift=0):
    for l in lags:
        df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)

def calculate_rollings(df, col, rollings, shift=0):
    for r in rollings:
        df[f"rol_mean_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).mean()
        df[f"rol_std_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).std()
        df[f"rol_min_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).min()
        df[f"rol_max_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).max()

def calculate_agg(df, agg_col, group_col):
    group_col_name = "_".join(group_col)
    df[f"mean_{agg_col}_by_{group_col_name}"] = df.groupby(group_col)[agg_col].transform("mean")
    df[f"std_{agg_col}_by_{group_col_name}"] = df.groupby(group_col)[agg_col].transform("std")
    


In [50]:
lags_target=list(range(1, 104))
rollings_target=[2, 4, 8, 16, 26, 52, 104]

horizon = 16

TARGET_COL = "sales"

agg_cols =  [
            ['stateid'],
            ['bu'],
            ['cityid'],
            ['famid'],
            ['stateid', 'famid'],
            ['stateid', 'cityid'],
            ['stateid', 'cluster'],
            ['cluster', 'bu'],
            ['cluster']
            ]

In [51]:
list_cutoff = ['2017-07-31']

In [52]:
for cutoff in tqdm(list_cutoff):
    
    print(cutoff)
    d_cutoff = pd.to_datetime(cutoff)
    day_max = d_cutoff +  datetime.timedelta(days=horizon -1)
    df_cutoff = df[df['date'] <= day_max]
    df_cutoff.loc[df_cutoff['date'] >= d_cutoff, 'sales'] = np.NaN
    print(df_cutoff.date.max())
    
    outdir = path + f'/data/fe/cutoff/{cutoff}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    df_cutoff.to_csv(f'data/fe/cutoff/{cutoff}/sales_cutoff.csv', index=False, sep=';')
    
    sales_df = df_cutoff[['ts_id', 'bu', 'famid','date', 'sales']]
    
    calculate_lags(sales_df, TARGET_COL, lags_target)
    calculate_rollings(sales_df, TARGET_COL, rollings_target)
    
    sales_df = sales_df.merge(df_store_transactions[['date', 'bu', 'cityid', 'stateid', 'cluster']],
                              how='left', on=['date', 'bu'])
    
    for group_col in agg_cols:
        calculate_agg(sales_df, TARGET_COL, group_col)
    
    
    sales_df = sales_df.drop(['bu', 'famid', 'cityid', 'stateid', 'cluster'], axis=1)
    sales_df["is_future"] = sales_df["sales"].isnull()

    # reduce df_features droping NA in the past
    sales_df = pd.concat([
        sales_df[~sales_df["is_future"]].dropna(),
        sales_df[sales_df["is_future"]]
    ])

    sales_df["time_idx"] = ((sales_df["date"] - sales_df["date"].min()).dt.days).astype(int)

    first_future_time_idx = sales_df.loc[sales_df["is_future"], "time_idx"].min()
    sales_df["forecast_step"] = sales_df["time_idx"] - first_future_time_idx + 1
    
    sales_df.to_csv(f'data/fe/cutoff/{cutoff}/df_festures.csv', index=False, sep=';')
    

  0%|                                                     | 0/1 [00:00<?, ?it/s]

2017-07-31
2017-08-15 00:00:00


/var/folders/4n/2ycm0xd14jgb1blvw6f3mzvr0000gn/T/ipykernel_939/104598143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)
/var/folders/4n/2ycm0xd14jgb1blvw6f3mzvr0000gn/T/ipykernel_939/104598143.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)
/var/folders/4n/2ycm0xd14jgb1blvw6f3mzvr0000gn/T/ipykernel_939/104598143.py:3: PerformanceWarning: Da

In [53]:
sales_df.head()

,ts_id,date,sales,lag_sales_1,lag_sales_2,lag_sales_3,lag_sales_4,lag_sales_5,lag_sales_6,lag_sales_7,...,std_sales_by_stateid_cityid,mean_sales_by_stateid_cluster,std_sales_by_stateid_cluster,mean_sales_by_cluster_bu,std_sales_by_cluster_bu,mean_sales_by_cluster,std_sales_by_cluster,is_future,time_idx,forecast_step
103,1_0,2013-04-15,1.0,0.0,0.0,3.0,5.0,3.0,4.0,3.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,False,1,-1567
104,1_0,2013-04-16,3.0,1.0,0.0,0.0,3.0,5.0,3.0,4.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,False,2,-1566
105,1_0,2013-04-17,4.0,3.0,1.0,0.0,0.0,3.0,5.0,3.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,False,3,-1565
106,1_0,2013-04-18,0.0,4.0,3.0,1.0,0.0,0.0,3.0,5.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,False,4,-1564
107,1_0,2013-04-19,4.0,0.0,4.0,3.0,1.0,0.0,0.0,3.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,False,5,-1563


In [54]:
sales_df[sales_df.ts_id == '1_0'].sort_values('date', ascending=False).isnull().sum()

ts_id                     0
date                      0
sales                    16
lag_sales_1              15
lag_sales_2              14
                         ..
mean_sales_by_cluster     0
std_sales_by_cluster      0
is_future                 0
time_idx                  0
forecast_step             0
Length: 155, dtype: int64

In [55]:
sales_df[sales_df.ts_id == '1_0'].sort_values('date', ascending=False).head(20)

,ts_id,date,sales,lag_sales_1,lag_sales_2,lag_sales_3,lag_sales_4,lag_sales_5,lag_sales_6,lag_sales_7,...,std_sales_by_stateid_cityid,mean_sales_by_stateid_cluster,std_sales_by_stateid_cluster,mean_sales_by_cluster_bu,std_sales_by_cluster_bu,mean_sales_by_cluster,std_sales_by_cluster,is_future,time_idx,forecast_step
1686,1_0,2017-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1584,16
1685,1_0,2017-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1583,15
1684,1_0,2017-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1582,14
1683,1_0,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1581,13
1682,1_0,2017-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1580,12
1681,1_0,2017-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1579,11
1680,1_0,2017-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1578,10
1679,1_0,2017-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1577,9
1678,1_0,2017-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1576,8
1677,1_0,2017-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1662.976645,413.425163,998.959234,288.148076,626.985139,391.75253,950.365401,True,1575,7
